# TODO

----------------> here
1. when get the df2, check the same elements


# Parameters

In [1]:
# starting url
param_url = "https://digitalcollections.frick.org/digico/#/"

# output image folder, excel file
input_excel_file = "../American_School_metadata_Stanford-csv1.csv"
OUTPUT_DIRECTORY = "D:/Penn/KosLab/Data/Frick/"
output_folder = "images/" # please create this folder before running this program
output_excel_file = "url.csv"

# fields
param_header = ["ID", "RECORD #"]

# image size
LARGE = 'LARGE'
MEDIUM = 'MEDIUM'
param_image_size = LARGE

In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd
import numpy as np
import re
# import wget  # use requests instead for https
import requests 
import os
# import xlrd
import hashlib

# Start program

In [11]:
if os.path.exists(OUTPUT_DIRECTORY + output_excel_file): 
    print("Output CSV exists")
    df2 = pd.read_csv(OUTPUT_DIRECTORY + output_excel_file)
else:
    print("No output CSV")
    # Read input CSV
    df1 = pd.read_csv(input_excel_file)
    num = len(df1)
    print(num)
    # Create output CSV df
    df2 = pd.DataFrame(columns=param_header)
    df2 = df1[param_header]
    df2["FILE NAME"] = ""
    df2["IMG_URL"] = ""

df2

Output CSV exists


,Unnamed: 0,ID,RECORD #,FILE NAME,IMG_URL
0,0,1,b11843731,"['b11843731-1.jpg', 'b11843731-2.jpg']",['https://digitalcollections.frick.org/media/i...
1,1,2,b10076360,['b10076360-1.jpg'],['https://digitalcollections.frick.org/media/i...
2,2,3,b10126971,"['b10126971-1.jpg', 'b10126971-2.jpg']",['https://digitalcollections.frick.org/media/i...
3,3,4,b10130135,"['b10130135-1.jpg', 'b10130135-2.jpg']",['https://digitalcollections.frick.org/media/i...
4,4,5,b1013119x,"['b1013119x-1.jpg', 'b1013119x-2.jpg']",['https://digitalcollections.frick.org/media/i...
5,5,6,b10131255,"['b10131255-1.jpg', 'b10131255-2.jpg']",['https://digitalcollections.frick.org/media/i...
6,6,7,b10131528,"['b10131528-1.jpg', 'b10131528-2.jpg']",['https://digitalcollections.frick.org/media/i...
7,7,8,b10131565,"['b10131565-1.jpg', 'b10131565-2.jpg']",['https://digitalcollections.frick.org/media/i...
8,8,9,b10132612,"['b10132612-1.jpg', 'b10132612-2.jpg']",['https://digitalcollections.frick.org/media/i...
9,9,10,b10132624,"['b10132624-1.jpg', 'b10132624-2.jpg']",['https://digitalcollections.frick.org/media/i...


In [4]:
# Start selenium
driver = webdriver.Chrome('../chromedriver')
driver.get(param_url)


### Function: count existing images

In [5]:
def get_existing_img_count(OUTPUT_DIRECTORY, output_folder):
    files = os.listdir(OUTPUT_DIRECTORY + output_folder)
    return len(files)

### Extract url

In [6]:
for i in range(len(df2)):
# for i in range(40000, len(df2)):
    if not pd.isna(df2.loc[i, "IMG_URL"]):
#     if df2.loc[i, "IMG_URL"] != "":
        continue

    record_num = df2["RECORD #"][i]
    driver.find_element_by_class_name("search-box").send_keys(record_num)
    driver.find_element_by_xpath("//button[contains(@class, 'search-button') and contains(@ng-click, 'textSearch()')]").click()
    time.sleep(3)
    driver.find_element_by_class_name("search-box").clear()

    cards = driver.find_elements_by_xpath("//div[contains(@class, 'document') and contains(@class, 'ng-scope') and contains(@ng-repeat, 'dochit')]")
    print(str(i) + ": " + str(len(cards)))
    if len(cards) == 0:
        print("Record number ", record_num, " matched no results")
        df2.at[i, "FILE NAME"] = "-1"
        df2.at[i, "IMG_URL"] = "-1"
        time.sleep(1)
        continue
    time.sleep(1)
    url_cards = cards[0].find_elements_by_xpath("//a[contains(text(),'Large')]")
    time.sleep(1)
    count = 1
    imgname_list = []
    url_list = []
    for url_card in url_cards:
        img_name = str(record_num) + "-" + str(count) + ".jpg"
        img_url = url_card.get_attribute("href")
        imgname_list.append(img_name)
        url_list.append(img_url)
        count = count + 1

    #     print(url_list)
    df2.at[i, "FILE NAME"] = imgname_list
    df2.at[i, "IMG_URL"] = url_list

    if i%50 == 0:
        # save to excel
        print("Saving to csv...")
        df2.to_csv(OUTPUT_DIRECTORY + output_excel_file) 
    

19951: 1
19952: 2
19953: 1
19954: 1
19955: 1
19956: 2
19957: 2
19958: 2
19959: 1
19960: 1
19961: 2
19962: 2
19963: 2
19964: 1
19965: 2
19966: 2
19967: 2
19968: 2
19969: 1
19970: 1
19971: 2
19972: 1
19973: 1
19974: 1
19975: 1
19976: 1
19977: 1
19978: 2
19979: 2
19980: 3
19981: 2
19982: 2
19983: 1
19984: 2
19985: 2
19986: 2
19987: 2
19988: 2
19989: 2
19990: 2
19991: 2
19992: 2
19993: 2
19994: 2
19995: 2
19996: 2
19997: 2
19998: 2
19999: 1
39951: 1
39952: 1
39953: 1
39954: 1
39955: 1
39956: 1
39957: 1
39958: 1
39959: 1
39960: 1
39961: 0
Record number  b13312716  matched no results
39962: 1
39963: 1
39964: 1
39965: 1
39966: 1
39967: 1
39968: 1
39969: 0
Record number  b13312790  matched no results
39970: 1
39971: 1
39972: 1
39973: 1
39974: 1
39975: 1
39976: 1
39977: 1
39978: 1
39979: 1
39980: 1
39981: 1
39982: 1
39983: 1
39984: 1
39985: 1
39986: 1
39987: 0
Record number  b13312984  matched no results
39988: 1
39989: 1
39990: 1
39991: 1
39992: 1
39993: 1
39994: 1
39995: 1
39996: 1
39997: 1
3

In [7]:
# save to excel
df2.to_csv(OUTPUT_DIRECTORY + output_excel_file)

### Function: download images

In [12]:
def download_images(df, seconds):
    print("Download images...")
    # download images
    for index, row in df.iterrows():
        if df2.loc[index, "IMG_URL"] == "-1":
            continue
        if pd.isna(df2.loc[index, "IMG_URL"]):
            continue
        filenames = row["FILE NAME"]
        urls = row["IMG_URL"]
        
        filenames = filenames.replace('[', '')
        filenames = filenames.replace(']', '')
        filenames = filenames.replace("'", "")
        filenames = filenames.replace(" ", "")
        filename_list = filenames.split(',')
        
        urls = urls.replace('[', '')
        urls = urls.replace(']', '')
        urls = urls.replace("'", "")
        urls = urls.replace(" ", "")
        url_list = urls.split(',')

        id = 0
        for filename in filename_list:
            if not os.path.exists(OUTPUT_DIRECTORY + output_folder + filename):
                print(filename, "Not exists")
                try:
                    pull_image = requests.get(url_list[id])
                    with open(OUTPUT_DIRECTORY + output_folder + filename, "wb+") as myfile:
                        myfile.write(pull_image.content)
                    time.sleep(seconds) # this is for being a polite visitor
#                 except Exception as e:
#                     print(e)
                except:
                    print("Can't download", filename, "using:", url_list[id])
            id = id + 1


# def delete_na(df, cols):
#     df = df.dropna(subset=cols)
#     df = df.reset_index(drop=True)
#     return df


In [14]:
download_images(df2, 4)

Download images...
